# 2022-81

Assess differences in population between DS ID and synthetic persons table
- DS mgra13_based_input tables
- synthetic persons tables == XPEF XX

In [1]:
import pandas as pd
import numpy as np
import pyodbc

## Data Preparation

In [2]:
dict_ds_xpef = {
    'DS35' : 'XPEF31',
    'DS38' : 'XPEF33',
    #'DS39' : 'XPEF34',
    'DS41' : 'XPEF35',
    'DS42' : 'XPEF36'
}

ds_file_base = 'J:/DataScience/DataQuality/QAQC/forecast_automation/mgra_series_13_outputs/aggregated_data/'

conn = pyodbc.connect('Driver={ODBC Driver 17 for SQL Server};'
                      'Server=ddamwsql16.sandag.org;'
                      'Database=isam;'
                      'Trusted_Connection=yes;')

In [16]:
csv = ds_file_base + 'mgra_DS42_ind_QA.csv'

df = pd.read_csv(csv, usecols=['mgra', 'year', 'hhp'])


In [15]:
df_sum = df.groupby(['year'])['mgra'].agg(['count', 'nunique'])
df_sum

,count,nunique
year,,
2016,23002,23002
2018,23002,23002
2020,23002,23002
2023,23002,23002
2025,23002,23002
2026,23002,23002
2029,23002,23002
2030,23002,23002
2032,23002,23002


In [18]:
df_sum = df.groupby(['year'])['hhp'].agg(['sum'])
df_sum

,sum
year,
2016,3203210
2018,3237268
2020,3263592
2023,3309666
2025,3345075
2026,3362198
2029,3410379
2030,3425282
2032,3453451


## ~~Check w/o GQ~~

SQL filter on [abm_syn_households] for [unittype] = 0

In [2]:
i = 0

for key in dict_ds_xpef:
    ## GRAB DS DATA
    ds_file = ds_file_base + 'mgra_' + key + '_ind_QA.csv'
    df_ds = pd.read_csv(ds_file, usecols=['mgra', 'year', 'hhp'])

    hhp_ds = 'hhp_' + key

    df_ds = df_ds.rename(columns={'hhp':hhp_ds})

    ## GRAB XPEF DATA
    xpef = dict_ds_xpef.get(key)
    
    query = ''.join(['''
                        SELECT [yr]-1 as 'year' -- [yr] is 1 year ahead?
                        ,[mgra]
                        ,SUM([persons]) as [tot_persons_''', xpef, ''']
                        FROM [isam].[''', xpef, '''].[abm_syn_households]
                        WHERE [unittype] = 0 -- filter for non-GQ
                              AND [yr] != '2018' -- 2017 does not exist in T drive files
                        GROUP BY [yr], [mgra]
                        ORDER BY [mgra], [yr]
                    '''])

    df_xpef = pd.read_sql_query(query, conn)

    ## MERGE DATAFRAMES
    if i == 0:
        df_merge = pd.merge(df_ds, df_xpef, on=['mgra', 'year'])
        i = 1
    else:
        df_merge = pd.merge(df_merge, df_ds, on=['mgra', 'year'])
        df_merge = pd.merge(df_merge, df_xpef, on=['mgra', 'year'])
    
    xpef_tot = 'tot_persons_' + xpef
    diff = dict_ds_xpef.get(key) + '-' + key
    df_merge[diff] = df_merge[xpef_tot] - df_merge[hhp_ds]

df_merge.sort_values(by=['mgra', 'year']).head()

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAP

,mgra,hhp_DS35,year,tot_persons_XPEF31,XPEF31-DS35,hhp_DS38,tot_persons_XPEF33,XPEF33-DS38,hhp_DS39,tot_persons_XPEF34,XPEF34-DS39,hhp_DS41,tot_persons_XPEF35,XPEF35-DS41,hhp_DS42,tot_persons_XPEF36,XPEF36-DS42
0,1,41,2016,41,0,41,41,0,41,41,0,41,41,0,41,41,0
17831,1,53,2018,53,0,41,41,0,52,52,0,50,50,0,44,44,0
35622,1,49,2020,49,0,47,47,0,43,43,0,43,43,0,44,44,0
53436,1,47,2023,47,0,38,38,0,42,42,0,41,41,0,46,46,0
71314,1,40,2025,40,0,35,35,0,42,42,0,42,42,0,45,45,0


In [ ]:
df_merge.to_csv('C:/Users/dle/Downloads/syn_persons and mgra input_diff.csv', index=False)
df_merge.head()

,mgra,hhp_DS35,year,tot_persons_XPEF31,XPEF31-DS35,hhp_DS38,tot_persons_XPEF33,XPEF33-DS38,hhp_DS39,tot_persons_XPEF34,XPEF34-DS39,hhp_DS41,tot_persons_XPEF35,XPEF35-DS41,hhp_DS42,tot_persons_XPEF36,XPEF36-DS42
0,1,41,2016,41,0,41,41,0,41,41,0,41,41,0,41,41,0
1,2,81,2016,81,0,81,81,0,81,81,0,81,81,0,81,81,0
2,3,111,2016,111,0,111,111,0,111,111,0,111,111,0,111,111,0
3,4,73,2016,73,0,73,73,0,73,73,0,73,73,0,73,73,0
4,5,63,2016,63,0,63,63,0,63,63,0,63,63,0,63,63,0


## ~~Check with GQ~~

No SQL filter on [abm_syn_households] to include GQ

In [3]:
i = 0

for key in dict_ds_xpef:
    ## GRAB DS DATA
    ds_file = ds_file_base + 'mgra_' + key + '_ind_QA.csv'
    df_ds = pd.read_csv(ds_file, usecols=['mgra', 'year', 'pop'])

    pop_ds = 'pop_' + key

    df_ds = df_ds.rename(columns={'hhp':hhp_ds})

    # Reorganize columns
    df_ds = df_ds[['mgra', 'year', pop_ds]]

    ## GRAB XPEF DATA
    xpef = dict_ds_xpef.get(key)
    
    query = ''.join(['''
                        SELECT [yr]-1 as 'year' -- [yr] is 1 year ahead?
                        ,[mgra]
                        ,SUM([persons]) as [tot_persons_''', xpef, ''']
                        FROM [isam].[''', xpef, '''].[abm_syn_households]
                        WHERE [yr] != '2018' -- 2017 does not exist in T drive files
                        GROUP BY [yr], [mgra]
                        ORDER BY [mgra], [yr]
                    '''])

    df_xpef = pd.read_sql_query(query, conn)

    ## MERGE DATAFRAMES
    if i == 0:
        df_merge_GQ = pd.merge(df_ds, df_xpef, on=['mgra', 'year'])
        i = 1
    else:
        df_merge_GQ = pd.merge(df_merge_GQ, df_ds, on=['mgra', 'year'])
        df_merge_GQ = pd.merge(df_merge_GQ, df_xpef, on=['mgra', 'year'])
    
    xpef_tot = 'tot_persons_' + xpef
    diff = dict_ds_xpef.get(key) + '-' + key
    df_merge_GQ[diff] = df_merge_GQ[xpef_tot] - df_merge_GQ[pop_ds]

df_merge_GQ = df_merge_GQ.sort_values(by=['mgra', 'year']).head()

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,mgra,hhp_DS42,year,tot_persons_XPEF36,XPEF36-DS42
0,1,41,2016,41,0
17854,1,44,2018,44,0
35704,1,44,2020,44,0
53599,1,46,2023,46,0
71722,1,45,2025,45,0


In [4]:
df_issue = df_merge_GQ.loc[df_merge_GQ['XPEF36-DS42'] > 0]
df_issue.mgra.unique()

,mgra,hhp_DS42,year,tot_persons_XPEF36,XPEF36-DS42
977,1065,444,2016,637,193
981,1070,438,2016,616,178
982,1071,41,2016,72,31
983,1073,1408,2016,1457,49
984,1076,0,2016,588,588
...,...,...,...,...,...
234586,19191,590,2050,3284,2694
234603,19214,0,2050,566,566
236907,22593,0,2050,577,577
236993,22705,0,2050,1074,1074


In [5]:
df_merge_GQ.to_csv('C:/Users/dle/Downloads/syn_persons and mgra input_diff GQ.csv', index=False)
df_merge_GQ.head()

,mgra,hhp_DS42,year,tot_persons_XPEF36,XPEF36-DS42
0,1,41,2016,41,0
1,2,81,2016,81,0
2,3,111,2016,111,0
3,4,73,2016,73,0
4,5,63,2016,63,0


## Check w/ all GQ, non-GQ, total pop

SQL query includes CASE statement to total based on GQ/non-GQ/total pop

In [9]:
i = 0

for key in dict_ds_xpef:
    ## GRAB DS DATA
    ds_file = ds_file_base + 'mgra_' + key + '_ind_QA.csv'
    df_ds = pd.read_csv(ds_file, usecols=['mgra', 'yr_id', 'Household Population', 'Group Quarters - College', 'Group Quarters - Military', 'Group Quarters - Other'])
    #df_ds['gq'] = df_ds['pop'] - df_ds['hhp']
    df_ds['gq'] = df_ds['Group Quarters - College'] + df_ds['Group Quarters - Military'] + df_ds['Group Quarters - Other']
    df_ds['pop'] = df_ds['gq'] + df_ds['Household Population']

    pop_ds = 'pop_mgra_summary_' + key
    hhp_ds = 'hhpop_mgra_summary_' + key
    gq_ds = 'gqpop_mgra_summary_' + key

    df_ds = df_ds.rename(columns={
        'yr_id':'year',
        'pop':pop_ds,
        'Household Population':hhp_ds,
        'gq': gq_ds})

    # Reorganize columns
    df_ds = df_ds[['mgra', 'year', gq_ds, hhp_ds, pop_ds]]

    ## GRAB XPEF DATA
    xpef = dict_ds_xpef.get(key)
    
    query = ''.join(['''
                        SELECT [yr]-1 as 'year' -- [yr] is 1 year ahead?
                        ,[mgra]
                        ,SUM(CASE WHEN [unittype] = 1 THEN [persons] END) as [gqpop_syn_persons_''', key, ''']
                        ,SUM(CASE WHEN [unittype] = 0 THEN [persons] END) as [hhpop_syn_persons_''', key, ''']
                        ,SUM([persons]) as [pop_syn_persons_''', key, ''']
                        FROM [isam].[''', xpef, '''].[abm_syn_households]
                        WHERE [yr] != '2018' -- 2017 does not exist in T drive files
                        GROUP BY [yr], [mgra]
                        ORDER BY [mgra], [yr]
                    '''])

    df_xpef = pd.read_sql_query(query, conn)
    

    ## MERGE DATAFRAMES
    if i == 0:
        df_merge_all = pd.merge(df_ds, df_xpef, on=['mgra', 'year'])
        i = 1
    else:
        df_merge_all = pd.merge(df_merge_all, df_ds, on=['mgra', 'year'])
        df_merge_all = pd.merge(df_merge_all, df_xpef, on=['mgra', 'year'])
    
    xpef_tot = 'pop_syn_persons_' + key
    diff = key + ' diff [syn-mgra]'
    df_merge_all[diff] = df_merge_all[xpef_tot] - df_merge_all[pop_ds]

df_merge_all = df_merge_all.sort_values(by=['mgra', 'year'])
df_merge_all.head()

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(
c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAP

,mgra,year,gqpop_mgra_summary_DS35,hhpop_mgra_summary_DS35,pop_mgra_summary_DS35,gqpop_syn_persons_DS35,hhpop_syn_persons_DS35,pop_syn_persons_DS35,DS35 diff [syn-mgra],gqpop_mgra_summary_DS38,...,hhpop_syn_persons_DS41,pop_syn_persons_DS41,DS41 diff [syn-mgra],gqpop_mgra_summary_DS42,hhpop_mgra_summary_DS42,pop_mgra_summary_DS42,gqpop_syn_persons_DS42,hhpop_syn_persons_DS42,pop_syn_persons_DS42,DS42 diff [syn-mgra]
0,1,2016,0,41,41,NaN,41.0,41,0,0,...,41.0,41,0,0,41,41,NaN,41.0,41,0
1,1,2018,0,53,53,NaN,53.0,53,0,0,...,50.0,50,0,0,44,44,NaN,44.0,44,0
2,1,2020,0,49,49,NaN,49.0,49,0,0,...,43.0,43,0,0,44,44,NaN,44.0,44,0
3,1,2025,0,40,40,NaN,40.0,40,0,0,...,42.0,42,0,0,45,45,NaN,45.0,45,0
4,1,2030,0,47,47,NaN,47.0,47,0,0,...,42.0,42,0,0,41,41,NaN,41.0,41,0


In [11]:
# Export to CSV
df_merge_all.to_csv('C:/Users/dle/Downloads/syn_persons and mgra input_diff all v2.csv', index=False)

## Investigate GQ impact

Finding: housing_type_id = 4 (GQ Other) is causing the diff

In [16]:
# Query for GQ Other (housing_type_id = 4)

query = ''' 
        WITH tbl1 AS
            (
                SELECT mgra, yr_id, datasource_id, [population] as pop
                FROM [demographic_warehouse].[fact].[population] a
                    INNER JOIN [demographic_warehouse].[dim].[mgra_denormalize] b
                ON a.mgra_id= b.mgra_id
                WHERE datasource_id IN (35, 38, 41, 42) AND housing_type_id=4 AND population <> 0
            )

        SELECT mgra
            ,yr_id as year
            ,[35] as gq_other_DS35
            ,[38] as gq_other_DS38
            ,[41] as gq_other_DS41
            ,[42] as gq_other_DS42
        FROM tbl1
        PIVOT (SUM(pop) FOR datasource_id IN ([35], [38],[41],[42])) p
        ORDER BY mgra, yr_id ASC'''

df_gq = pd.read_sql_query(query, conn)

df_gq.head()

c:\Users\dle\Anaconda3\lib\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,mgra,year,gq_other_DS35,gq_other_DS38,gq_other_DS41,gq_other_DS42
0,121,2016,6,6,6,6
1,121,2018,6,6,6,6
2,121,2020,6,6,6,6
3,121,2025,6,6,6,6
4,121,2030,6,6,6,6


In [15]:
# Export GQ Other to CSV
df_gq.to_csv('C:/Users/dle/Downloads/syn_persons and mgra input diff - GQ.csv', index=False)

In [13]:
# Optional merge of all dfs - the GQ Other data was merged separately via Power Query in the workbook
df_merge_all_GQ = pd.merge(df_merge_all, df_gq, how = 'left', on=['mgra', 'year'])
df_merge_all_GQ.head()

,mgra,year,gqpop_mgra_summary_DS35,hhpop_mgra_summary_DS35,pop_mgra_summary_DS35,gqpop_syn_persons_DS35,hhpop_syn_persons_DS35,pop_syn_persons_DS35,DS35 diff [syn-mgra],gqpop_mgra_summary_DS38,...,hhpop_mgra_summary_DS42,pop_mgra_summary_DS42,gqpop_syn_persons_DS42,hhpop_syn_persons_DS42,pop_syn_persons_DS42,DS42 diff [syn-mgra],gq_DS35,gq_DS38,gq_DS41,gq_DS42
0,1,2016,0,41,41,NaN,41.0,41,0,0,...,41,41,NaN,41.0,41,0,NaN,NaN,NaN,NaN
1,1,2018,0,53,53,NaN,53.0,53,0,0,...,44,44,NaN,44.0,44,0,NaN,NaN,NaN,NaN
2,1,2020,0,49,49,NaN,49.0,49,0,0,...,44,44,NaN,44.0,44,0,NaN,NaN,NaN,NaN
3,1,2025,0,40,40,NaN,40.0,40,0,0,...,45,45,NaN,45.0,45,0,NaN,NaN,NaN,NaN
4,1,2030,0,47,47,NaN,47.0,47,0,0,...,41,41,NaN,41.0,41,0,NaN,NaN,NaN,NaN


In [14]:
df_merge_all_GQ.to_csv('C:/Users/dle/Downloads/syn_persons and mgra input diff - GQ.csv', index=False)